In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.window import Window
import pandas as pd

DB_FILEPATH = r'C:\dados-tcs-itau\4 - ecommerce\data\ecommerce.db'

# Cria uma sessão do PySpark
spark: SparkSession = (
    SparkSession
    .builder
    .enableHiveSupport() # type: ignore
    .config('spark.jars.packages', 'org.xerial:sqlite-jdbc:3.34.0') # type: ignore
    .master('local[*]')
    .getOrCreate()
)

In [4]:
# Função auxiliar para ler tabelas do SQLite
def ler_tabela(nome: str) -> pd.DataFrame:
    return (
        spark.read
    .format('jdbc')
    .option('url', f'jdbc:sqlite:{DB_FILEPATH}')
    .option('driver', 'org.sqlite.JDBC')
    .option('dbtable', nome) # Lê a tabela inteira
    #.option('query', 'SELECT * FROM user') # Retorna o resultado da consulta SQL
    .load()
    )

# Lê as tabelas do banco
usuarios = ler_tabela("user")
pedidos = ler_tabela("'order'")
itens = ler_tabela("order_item")
produtos = ler_tabela("product")

In [5]:
# Filtra pedidos com status válidos e traduz status
pedidos_filtrados = pedidos.filter(
    F.col("status").isin("approved", "shipped", "delivered")
).withColumn(
    "status_traduzido",
    F.when(F.col("status") == "approved", "APROVADO")
    .when(F.col("status") == "shipped", "ENVIADO")
    .when(F.col("status") == "delivered", "ENTREGUE")
)

pedidos_filtrados.show()

+---+-------+-------------------+---------+-----------+-------------------+----------------+
| id|user_id|         order_date|   status|total_value|      creation_date|status_traduzido|
+---+-------+-------------------+---------+-----------+-------------------+----------------+
|  1|      1|2025-01-14 16:23:59|  shipped|    2362.35|2025-04-09 16:23:59|         ENVIADO|
|  3|      1|2024-06-15 16:23:59|delivered|    1938.32|2025-04-09 16:23:59|        ENTREGUE|
|  4|      1|2024-10-01 16:23:59|  shipped|    3670.62|2025-04-09 16:23:59|         ENVIADO|
|  6|      1|2024-08-06 16:23:59| approved|     6138.1|2025-04-09 16:23:59|        APROVADO|
|  9|      1|2025-02-03 16:23:59| approved|      984.0|2025-04-09 16:23:59|        APROVADO|
| 10|      1|2025-03-05 16:23:59| approved|    3722.28|2025-04-09 16:23:59|        APROVADO|
| 12|      1|2024-06-13 16:23:59| approved|    3044.49|2025-04-09 16:23:59|        APROVADO|
| 13|      1|2024-06-20 16:23:59| approved|    3475.34|2025-04-09 16:2

In [ ]:
#Unir tabelas e selecionar colunas necessárias
df_final = pedidos_filtrados.join(usuarios, pedidos_filtrados.user_id == usuarios.id, "left") \
    .join(itens, pedidos_filtrados.id == itens.order_id, "left") \
    .join(produtos, itens.product_id == produtos.id, "left") \
    .select(
        usuarios["name"].alias("nome_usuario"),
        usuarios["email"].alias("email_usuario"),
        pedidos_filtrados["id"].alias("id_pedido"),
        pedidos_filtrados["order_date"].alias("data_pedido"),
        produtos["name"].alias("nome_produto"),
        itens["quantity"].alias("quantidade_comprada"),
        itens["unit_price"].alias("preco_unitario"),
        produtos["category"].alias("categoria_produto"),
        pedidos_filtrados["total_value"].alias("total_gasto")
    )
    
df_final.show()

+---------------+--------------------+---------+-------------------+----------------+-------------------+--------------+-----------------+-----------+
|   nome_usuario|       email_usuario|id_pedido|        data_pedido|    nome_produto|quantidade_comprada|preco_unitario|categoria_produto|total_gasto|
+---------------+--------------------+---------+-------------------+----------------+-------------------+--------------+-----------------+-----------+
|Deborah Ramirez|joserodriguez@exa...|        1|2025-01-14 16:23:59|  Road newspaper|                  3|        787.45|           Sports|    2362.35|
|Deborah Ramirez|joserodriguez@exa...|        3|2024-06-15 16:23:59|Specific article|                  2|        969.16|           Sports|    1938.32|
|Deborah Ramirez|joserodriguez@exa...|        4|2024-10-01 16:23:59|  Road newspaper|                  3|        873.54|           Sports|    3670.62|
|Deborah Ramirez|joserodriguez@exa...|        4|2024-10-01 16:23:59|Specific article|         

In [ ]:
#Criar ranking com base no total gasto por usuário
window_spec = Window.partitionBy("nome_usuario").orderBy(F.col("total_gasto").desc())
df_final = df_final.withColumn("ranking_usuario", F.rank().over(window_spec))

#Exibir os primeiros registros
df_final.show()

+------------+--------------------+---------+-------------------+-----------------+-------------------+--------------+-----------------+-----------+---------------+
|nome_usuario|       email_usuario|id_pedido|        data_pedido|     nome_produto|quantidade_comprada|preco_unitario|categoria_produto|total_gasto|ranking_usuario|
+------------+--------------------+---------+-------------------+-----------------+-------------------+--------------+-----------------+-----------+---------------+
| Brian Ortiz|christina00@examp...|      718|2024-09-06 11:07:49|     City machine|                  2|         44.17|           Sports|    5824.59|              1|
| Brian Ortiz|christina00@examp...|      718|2024-09-06 11:07:49|  Participant boy|                  1|         910.5|             Food|    5824.59|              1|
| Brian Ortiz|christina00@examp...|      718|2024-09-06 11:07:49|     Though watch|                  5|        965.15|             Food|    5824.59|              1|
| Brian Or

In [ ]:
#Salvar como Parquet para uso posterior
df_final.write.mode("overwrite").parquet("C:/dados-tcs-itau/4 - ecommerce/data/resultados/ranking_usuarios.parquet")

In [ ]:
#Definir caminho do arquivo Parquet
RESULTADO_PATH = r"C:\dados-tcs-itau\4 - ecommerce\data\resultados\ranking_usuarios.parquet"

#Ler o arquivo Parquet
df_resultado = spark.read.parquet(RESULTADO_PATH)

#Se quiser filtrar os TOP 5 usuários com maior gasto:
df_top_5 = df_resultado.orderBy("ranking_usuario").limit(5)
df_top_5.show()

+---------------+--------------------+---------+-------------------+---------------+-------------------+--------------+-----------------+-----------+---------------+
|   nome_usuario|       email_usuario|id_pedido|        data_pedido|   nome_produto|quantidade_comprada|preco_unitario|categoria_produto|total_gasto|ranking_usuario|
+---------------+--------------------+---------+-------------------+---------------+-------------------+--------------+-----------------+-----------+---------------+
|Deborah Ramirez|joserodriguez@exa...|      275|2024-04-23 16:23:59| Road newspaper|                  2|        906.13|           Sports|    10511.2|              1|
|     Lisa Lopez|  royray@example.org|      684|2024-04-22 11:07:49|        Risk or|                  3|        406.18|     Home & Decor|    5589.34|              1|
|Deborah Ramirez|joserodriguez@exa...|      275|2024-04-23 16:23:59|Participant boy|                  3|       1024.91|             Food|    10511.2|              1|
|   